In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Projects\\End-To-End-ML-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Projects\\End-To-End-ML-MLflow'

In [5]:
# 4. Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
# 5.Config Manager and src
from mlflow.constants import *
from mlflow.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
# 6.Components
# import the packages

import os
import urllib.request as request
import zipfile
from mlflow import logger
from mlflow.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            logger.info(f"file already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        function returns None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
# 7 Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-14 15:18:56,284: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-14 15:18:56,298: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-14 15:18:56,301: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-14 15:18:56,303: INFO: common: created directory at: artifacts]
[2025-02-14 15:18:56,304: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-14 15:18:57,793: INFO: 1394977229: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 22187
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d2e4c05416bd38248c3112ae3def3690f6cca26b3d9f2771ad96b17a190e951e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3F7A:B682B:E32EE:11F426:67AF1183
Accept-Ranges: bytes
Date: Fri, 14 F